In [2]:
#!/usr/bin/env python
# coding: utf-8



#Importing libraries

import pandas as pd
import numpy as np
import cv2
import tensorflow as tf
import keras
import h5py
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.models import Sequential, Model
from keras.layers import Conv2D, Flatten, MaxPool2D, Input, Dropout, Dense
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping, Callback, ModelCheckpoint
from keras.utils import to_categorical
from keras.optimizers import Adam
import os
from pathlib import Path
import glob
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
import cv2
import matplotlib.pyplot as plt
import time
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

print('Imported Libraries')

# In[2]:


# #Unzip the dataset
# member = ('../GarbageClassification/garbage-classification.zip')
# from zipfile import ZipFile
# with ZipFile(member, 'r') as zipObj:
#        # Extract all the contents of zip file in current directory
#     zipObj.extractall()
 


# In[3]:

print('Reading the data')
#Reading the data

member = ('../Datasets/Garbage classification/Garbage classification/')
catagories = os.listdir(member)
list_items = []
for cat in catagories:
    catagory_img = (member  + cat)
    #catagory_img.glob('*.jpeg')
    for _ in (glob.glob(catagory_img +'/'+'*.jpg')):
        list_items.append([cat, _])
    
#Convert list into dataframe

data = pd.DataFrame(list_items,columns = ['catagory', 'filepath'], index = None)
data = data.sample(frac=1).reset_index(drop=True)
data.head(5)
data.shape


# In[5]:


print('Splitting the dataset')
train_data = data[1:2000]
val_data = data[2001:2200]
test_data = data[2201:2527]



Imported Libraries
Reading the data
Splitting the dataset


In [3]:
#Normalizing the images

#Resizing the images
def display_normal(a, title1 = 'Original'):
    plt.imshow(a), plt.title(title1)
    plt.show()
    
def display(a,b, title1 = 'Original', title2 = 'Edited'):
    plt.subplot(121), plt.imshow(a), plt.title(title1)
    plt.subplot(122), plt.imshow(a), plt.title(title2)
    plt.show()

def adv_preprocessing(image):
    #loading images
    #Getting 3 images to work with

    preimgs = []
    img = cv2.imread(image, cv2.IMREAD_UNCHANGED)

    #Setting dimensions to resize
    height = 224
    width = 224
    
    dim = (width, height)
    res = cv2.resize(img, dim, interpolation = cv2.INTER_LINEAR)
    preimgs.append(res)
        
#Removing noise from image - Gaussian blur
    
    blurred_img = cv2.GaussianBlur(res, (5,5),0)
    preimgs.append(blurred_img)

    #Segmentation 
    #------------------------------------------------------------------
    image = res
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    ret,thresh = cv2.threshold(gray, 0,255,cv2.THRESH_BINARY+ cv2.THRESH_OTSU)
    
    #More noise removal
    #------------------------------------------------------------------
    kernal = np.ones((3,3), np.uint8)
    opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernal, iterations=2)
    
    #Sure background area
    sure_bg = cv2.dilate(opening, kernal, iterations = 3)
    
    #Finding foreground area
    dist_transform = cv2.distanceTransform(opening, cv2.DIST_L2, 5)
    ret, sure_fg = cv2.threshold(dist_transform, 0.7 * dist_transform.max(), 255, 0)
    
    # Finding unknown region
    sure_fg = np.uint8(sure_fg)
    unknown = cv2.subtract(sure_bg, sure_fg)
    
    #Seperating different objects with different backgrounds
    #Markers labelling
    ret, markers  = cv2.connectedComponents(sure_fg)
    #Add one to all labels so that sure background is 0 not 1
    markers = markers+1
    
    #Mark the unknown region with 0
    markers[unknown == 255] = 0
    
    markers = cv2.watershed(res, markers)
    res[markers == -1] = [255,0,0]
    placeholder = np.random.rand(224,224)
    #Displaying the markers on image
    markers = np.dstack([markers,np.zeros((224,224)), placeholder])
    #Adding 
    preimgs.append(res)
    preimgs.append(markers)
    
    return preimgs

In [6]:
def adv_gendata(data, batch_size):
    labelenc = LabelEncoder()
#     dict = {'plastic' : 1, 'glass' : 2, 'cardboard': 3, 'metal': 4, 'paper' : 5,'trash': 6}
    n = len(data)
    steps = n//batch_size
    data['labels'] = labelenc.fit_transform(data['catagory'])
    enc = OneHotEncoder(handle_unknown='ignore')
    enc_df = pd.DataFrame(enc.fit_transform(data[['labels']]).toarray())

    #Defining numpy array to contain label and image data
    
    batch_data = np.zeros((batch_size, 224,224,3), dtype = np.float32)
    batch_labels = np.zeros((batch_size, 6))
    indices = np.arange(n)

    c1 = 0
    i = 0
    #Initialize counter
    while True:
        np.random.shuffle(indices)
        next_batch = indices[(i*batch_size):(i+1)*batch_size]
        count = 0
        

        for j, idx in enumerate(next_batch):
            if count <= batch_size-4:
                img_name = data.iloc[idx]['filepath']
                aug_img = adv_preprocessing(data['filepath'].iloc[idx])
                encoded_label = enc_df.iloc[idx]
                batch_data[count+0] = aug_img[0]
                batch_labels[count+0] = encoded_label
                batch_data[count+1] = aug_img[1]
                batch_labels[count+1] = encoded_label
                batch_data[count+2] = aug_img[2]
                batch_labels[count+2] = encoded_label
                batch_data[count+3] = aug_img[3]
                batch_labels[count+3] = encoded_label
                count +=4
                c1 = c1+1
                
            else:
                count+=1

            if count==batch_size:
                i += 1
                break

        i+=1
        yield batch_data, batch_labels
    
    if i>=steps:
        i=0   

In [112]:
train_data_gen = adv_gendata(train_data,20)
batch_data, batch_labels = next(train_data_gen)

print(batch_labels)


0
4
8
12


c:\programdata\miniconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


16
[[0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0. 0.]]


In [5]:
def build_model():
    model = Sequential()
    input_size  = Input(shape = (224,224,3), name  =  'Input_Image')

    #Layer 1 - Deapth Layer 1,2
    x = Conv2D(64,(3,3), activation = 'relu', padding = 'same', name = 'ConvLayer1' )(input_size)
#     x = Conv2D(64,(3,3), activation = 'relu', padding = 'same', name = 'ConvLayer2' )(x)
    x = MaxPool2D((2,2), name = 'Maxpool1')(x)
    x = BatchNormalization(name = 'bn1')(x)

    #Layer 2 - Deapth layer 3,4
    x = Conv2D(128,(3,3), activation = 'relu', padding = 'same', name = 'ConvLayer3')(x)
#     x = Conv2D(128,(3,3), activation = 'relu', padding = 'same', name = 'ConvLayer4')(x)
    x = MaxPool2D((2,2), name = 'Maxpoo12')(x)
    x = BatchNormalization(name = 'bn2')(x)
#     #Layer 3 - Deapth layer 3
#     x = Conv2D(30,(3,3), activation= 'relu',padding = 'same',  name = 'ConvLayer3')(x)
#     x = MaxPool2D((2,2), name = 'Maxpool3')(x)

    #Flatten the model

    x = Flatten(name = 'Flatten')(x)
    x = Dense(128, activation = 'relu', name = 'FC1')(x)
    x = Dense(128, activation = 'relu', name = 'FC2')(x)
    x = Dropout(0.7, name = 'Dropout2')(x)
    x = Dense(6, activation = 'softmax', name = 'Fc3')(x)
    
    model = Model(input = input_size , output = x)
    return model

#Building the model and summary

model = build_model() 
model.summary()

W0311 22:26:57.710695  4344 deprecation_wrapper.py:119] From c:\programdata\miniconda3\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0311 22:26:57.742697  4344 deprecation_wrapper.py:119] From c:\programdata\miniconda3\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0311 22:26:57.754695  4344 deprecation_wrapper.py:119] From c:\programdata\miniconda3\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0311 22:26:57.778776  4344 deprecation_wrapper.py:119] From c:\programdata\miniconda3\lib\site-packages\keras\backend\tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0311 22:26:57.794695  4344 deprecation_wrapper.py:119] From c:\programdata

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Input_Image (InputLayer)     (None, 224, 224, 3)       0         
_________________________________________________________________
ConvLayer1 (Conv2D)          (None, 224, 224, 64)      1792      
_________________________________________________________________
Maxpool1 (MaxPooling2D)      (None, 112, 112, 64)      0         
_________________________________________________________________
bn1 (BatchNormalization)     (None, 112, 112, 64)      256       
_________________________________________________________________
ConvLayer3 (Conv2D)          (None, 112, 112, 128)     73856     
_________________________________________________________________
Maxpoo12 (MaxPooling2D)      (None, 56, 56, 128)       0         
_________________________________________________________________
bn2 (BatchNormalization)     (None, 56, 56, 128)       512       
__________

c:\programdata\miniconda3\lib\site-packages\ipykernel_launcher.py:28: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("In..., outputs=Tensor("Fc...)`


In [6]:
opt = Adam(lr = 0.0001, decay = 1e-5)
es = EarlyStopping(patience=5)
chkpt = ModelCheckpoint(filepath= 'bestmodel',save_best_only=True, save_weights_only=True)
model.compile(loss= 'binary_crossentropy', metrics= ['accuracy'], optimizer= opt)


W0311 22:27:05.999522  4344 deprecation_wrapper.py:119] From c:\programdata\miniconda3\lib\site-packages\keras\optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0311 22:27:06.019493  4344 deprecation.py:323] From c:\programdata\miniconda3\lib\site-packages\tensorflow\python\ops\nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [81]:
batch_size = 20
epochs = 4
model.fit_generator(train_data_gen,epochs=epochs, verbose = 1, steps_per_epoch=20)


Epoch 1/4
20/20 [==============================] - 25s 1s/step - loss: 3.0960 - acc: 0.7417
Epoch 2/4
20/20 [==============================] - 13s 673ms/step - loss: 3.0461 - acc: 0.7396
Epoch 3/4
20/20 [==============================] - 13s 638ms/step - loss: 3.1350 - acc: 0.7383
Epoch 4/4
20/20 [==============================] - 12s 620ms/step - loss: 3.4418 - acc: 0.7283
